# Real-Time Market Data Integration Pipeline

## Overview

This notebook demonstrates how to integrate Python/FastMCP MCP servers as data sources for real-time market data ingestion. Connect to real-time market data MCP servers via URL, ingest live market data, and build a temporal trading knowledge graph.

**IMPORTANT**: This implementation supports ONLY Python-based MCP servers and FastMCP servers. Users can bring their own Python/FastMCP MCP servers via URL connections.

### Modules Used (20+)

- **Ingestion**: MCPIngestor, ingest_mcp, StreamIngestor
- **Parsing**: MCPParser, JSONParser, StructuredDataParser
- **Extraction**: NERExtractor, RelationExtractor, EventDetector, SemanticAnalyzer
- **KG**: GraphBuilder, TemporalGraphQuery, TemporalPatternDetector, GraphAnalyzer
- **Analytics**: CentralityCalculator, CommunityDetector, ConnectivityAnalyzer
- **Reasoning**: InferenceEngine, RuleManager, ExplanationGenerator
- **Export**: JSONExporter, CSVExporter, RDFExporter, ReportGenerator
- **Visualization**: KGVisualizer, TemporalVisualizer, AnalyticsVisualizer

### Pipeline

**Connect to Real-Time Market MCP Server → Ingest Live Market Data via MCP → Parse MCP Responses → Extract Trading Entities → Build Temporal Trading KG → Analyze Market Patterns → Generate Reports → Visualize**

---

## Step 1: Connect to Real-Time Market Data MCP Server

Connect to a Python/FastMCP MCP server that provides real-time market data via URL. The MCP server can expose resources (market feeds) and tools (real-time price queries, market data streams).


In [ ]:
from semantica.ingest import MCPIngestor, ingest_mcp, StreamIngestor
from semantica.parse import MCPParser, JSONParser, StructuredDataParser
from semantica.semantic_extract import NERExtractor, RelationExtractor, EventDetector, SemanticAnalyzer
from semantica.kg import GraphBuilder, TemporalGraphQuery, TemporalPatternDetector, GraphAnalyzer
from semantica.kg import CentralityCalculator, CommunityDetector, ConnectivityAnalyzer
from semantica.reasoning import InferenceEngine, RuleManager, ExplanationGenerator
from semantica.export import JSONExporter, CSVExporter, RDFExporter, ReportGenerator
from semantica.visualization import KGVisualizer, TemporalVisualizer, AnalyticsVisualizer
import json
from datetime import datetime, timedelta

# Initialize MCP ingestor
mcp_ingestor = MCPIngestor()

# Connect to real-time market data MCP server via URL
# Replace with your actual MCP server URL
# Example: http://localhost:8000/mcp or https://api.example.com/market-mcp
market_mcp_url = "http://localhost:8000/mcp"

try:
    # Connect to MCP server with authentication (if required)
    mcp_ingestor.connect(
        "market_server",
        url=market_mcp_url,
        headers={
            "Authorization": "Bearer your_token",
            "X-API-Key": "your_api_key"
        } if "api.example.com" in market_mcp_url else {}
    )
    print(f"✓ Connected to real-time market data MCP server at {market_mcp_url}")
    
    # List available resources (market feeds, price streams)
    resources = mcp_ingestor.list_available_resources("market_server")
    print(f"\n📊 Available Resources ({len(resources)}):")
    for resource in resources[:5]:  # Show first 5
        print(f"  - {resource.uri}: {resource.name}")
        if resource.description:
            print(f"    {resource.description[:80]}...")
    
    # List available tools (real-time queries, market data streams)
    tools = mcp_ingestor.list_available_tools("market_server")
    print(f"\n🔧 Available Tools ({len(tools)}):")
    for tool in tools[:5]:  # Show first 5
        print(f"  - {tool.name}: {tool.description or 'No description'}")
        
except Exception as e:
    print(f"⚠ Connection failed: {e}")
    print("Note: This example uses a placeholder URL. Replace with your actual MCP server URL.")
    print("For testing, you can use a mock MCP server or skip connection and use sample data below.")


## Step 2: Ingest Real-Time Market Data from MCP Server

Ingest live market data using both resource-based and tool-based methods, including streaming data.


In [ ]:
# Initialize parsers
mcp_parser = MCPParser()
json_parser = JSONParser()
structured_parser = StructuredDataParser()

market_data = []

# Method 1: Resource-based ingestion
# Ingest from MCP resources (market feeds)
try:
    # Example: Ingest real-time market feed resource
    market_feeds = mcp_ingestor.ingest_resources(
        "market_server",
        resource_uris=["resource://market/real_time", "resource://market/stream"]
    )
    
    for item in market_feeds:
        market_data.append(item)
        print(f"✓ Ingested resource: {item.resource_uri}")
        
except Exception as e:
    print(f"⚠ Resource ingestion: {e}")

# Method 2: Tool-based ingestion
# Call MCP tools to retrieve real-time data dynamically
try:
    # Example: Get real-time stock prices
    real_time_prices = mcp_ingestor.ingest_tool_output(
        "market_server",
        tool_name="get_real_time_prices",
        arguments={
            "symbols": ["AAPL", "MSFT", "GOOGL", "TSLA", "AMZN"],
            "interval": "1min"
        }
    )
    
    if real_time_prices:
        market_data.append(real_time_prices)
        print(f"✓ Retrieved real-time prices via tool")
        
    # Example: Get market depth data
    market_depth = mcp_ingestor.ingest_tool_output(
        "market_server",
        tool_name="get_market_depth",
        arguments={
            "symbol": "AAPL",
            "levels": 5
        }
    )
    
    if market_depth:
        market_data.append(market_depth)
        print(f"✓ Retrieved market depth via tool")
        
except Exception as e:
    print(f"⚠ Tool-based ingestion: {e}")
    print("Note: Using sample data for demonstration")

# Sample real-time market data (if MCP server is not available)
if not market_data:
    print("\n📝 Using sample real-time market data for demonstration:")
    sample_data = {
        "real_time_prices": [
            {
                "symbol": "AAPL",
                "price": 175.50,
                "change": 2.30,
                "change_percent": 1.33,
                "volume": 45000000,
                "timestamp": datetime.now().isoformat(),
                "bid": 175.48,
                "ask": 175.52,
                "spread": 0.04
            },
            {
                "symbol": "MSFT",
                "price": 380.25,
                "change": -1.50,
                "change_percent": -0.39,
                "volume": 28000000,
                "timestamp": datetime.now().isoformat(),
                "bid": 380.20,
                "ask": 380.30,
                "spread": 0.10
            },
            {
                "symbol": "GOOGL",
                "price": 142.80,
                "change": 3.20,
                "change_percent": 2.29,
                "volume": 32000000,
                "timestamp": datetime.now().isoformat(),
                "bid": 142.75,
                "ask": 142.85,
                "spread": 0.10
            },
            {
                "symbol": "TSLA",
                "price": 245.60,
                "change": 5.40,
                "change_percent": 2.25,
                "volume": 55000000,
                "timestamp": datetime.now().isoformat(),
                "bid": 245.55,
                "ask": 245.65,
                "spread": 0.10
            },
            {
                "symbol": "AMZN",
                "price": 155.30,
                "change": 1.20,
                "change_percent": 0.78,
                "volume": 42000000,
                "timestamp": datetime.now().isoformat(),
                "bid": 155.25,
                "ask": 155.35,
                "spread": 0.10
            }
        ]
    }
    market_data.append(sample_data)
    print(f"  Loaded {len(sample_data['real_time_prices'])} real-time price records")

print(f"\n📊 Total market data items ingested: {len(market_data)}")


## Step 3: Parse Real-Time Market Data

Parse the real-time market data received from MCP server responses.


In [ ]:
parsed_market_data = []

# Parse MCP responses
for data_item in market_data:
    try:
        # Parse MCP response (handles JSON, text, binary)
        if isinstance(data_item, dict):
            parsed_item = data_item
        else:
            parsed_item = mcp_parser.parse_response(data_item, response_type="json")
        
        parsed_market_data.append(parsed_item)
        
    except Exception as e:
        print(f"⚠ Parsing error: {e}")

# Extract real-time prices
real_time_prices = []

for item in parsed_market_data:
    if isinstance(item, dict):
        if "real_time_prices" in item:
            real_time_prices.extend(item["real_time_prices"])
        elif "symbol" in item and "price" in item:
            real_time_prices.append(item)

print(f"✓ Parsed {len(parsed_market_data)} data items")
print(f"✓ Extracted {len(real_time_prices)} real-time price records")


## Step 4: Extract Trading Entities and Relationships

Extract trading entities (stocks, prices, market events) and relationships from real-time MCP data.


In [ ]:
ner_extractor = NERExtractor()
relation_extractor = RelationExtractor()
event_detector = EventDetector()
semantic_analyzer = SemanticAnalyzer()

trading_entities = []
trading_relationships = []

# Extract from real-time prices
for price_data in real_time_prices:
    if isinstance(price_data, dict):
        symbol = price_data.get("symbol", "")
        timestamp = price_data.get("timestamp", "")
        
        # Stock entity with real-time price
        trading_entities.append({
            "id": f"{symbol}_{timestamp}",
            "type": "PricePoint",
            "name": f"{symbol} @ {timestamp}",
            "properties": {
                "symbol": symbol,
                "price": price_data.get("price", 0),
                "change": price_data.get("change", 0),
                "change_percent": price_data.get("change_percent", 0),
                "volume": price_data.get("volume", 0),
                "bid": price_data.get("bid", 0),
                "ask": price_data.get("ask", 0),
                "spread": price_data.get("spread", 0),
                "timestamp": timestamp
            }
        })
        
        # Stock entity
        trading_entities.append({
            "id": symbol,
            "type": "Stock",
            "name": symbol,
            "properties": {}
        })
        
        # PricePoint-Stock relationship
        trading_relationships.append({
            "source": f"{symbol}_{timestamp}",
            "target": symbol,
            "type": "price_for",
            "properties": {"timestamp": timestamp}
        })
        
        # Market event detection
        if price_data.get("change_percent", 0) > 2:
            event_id = f"event_{symbol}_{timestamp}"
            trading_entities.append({
                "id": event_id,
                "type": "MarketEvent",
                "name": "Price Surge",
                "properties": {
                    "event_type": "surge",
                    "magnitude": price_data.get("change_percent", 0),
                    "timestamp": timestamp
                }
            })
            trading_relationships.append({
                "source": event_id,
                "target": symbol,
                "type": "affects",
                "properties": {"timestamp": timestamp}
            })

# Remove duplicates
seen_entities = set()
unique_entities = []
for entity in trading_entities:
    entity_key = (entity["id"], entity["type"])
    if entity_key not in seen_entities:
        seen_entities.add(entity_key)
        unique_entities.append(entity)

trading_entities = unique_entities

print(f"✓ Extracted {len(trading_entities)} trading entities")
print(f"  - Price Points: {len([e for e in trading_entities if e['type'] == 'PricePoint'])}")
print(f"  - Stocks: {len([e for e in trading_entities if e['type'] == 'Stock'])}")
print(f"  - Market Events: {len([e for e in trading_entities if e['type'] == 'MarketEvent'])}")
print(f"✓ Extracted {len(trading_relationships)} relationships")


## Step 5: Build Temporal Trading Knowledge Graph

Build a temporal knowledge graph from the extracted trading entities and relationships.


In [ ]:
builder = GraphBuilder()
temporal_query = TemporalGraphQuery()
temporal_pattern_detector = TemporalPatternDetector()
graph_analyzer = GraphAnalyzer()

# Build temporal knowledge graph
trading_kg = builder.build(trading_entities, trading_relationships)

# Analyze graph structure
metrics = graph_analyzer.compute_metrics(trading_kg)
centrality_calculator = CentralityCalculator()
community_detector = CommunityDetector()
connectivity_analyzer = ConnectivityAnalyzer()

# Calculate graph metrics
centrality_scores = centrality_calculator.calculate_centrality(trading_kg, measure="degree")
communities = community_detector.detect_communities(trading_kg)
connectivity = connectivity_analyzer.analyze_connectivity(trading_kg)

# Detect temporal patterns
temporal_patterns = temporal_pattern_detector.detect_temporal_patterns(
    trading_kg,
    pattern_type="trend",
    min_frequency=1
)

print(f"✓ Built temporal trading knowledge graph")
print(f"  Entities: {len(trading_kg.get('entities', []))}")
print(f"  Relationships: {len(trading_kg.get('relationships', []))}")
print(f"  Graph density: {metrics.get('density', 0):.3f}")
print(f"  Communities detected: {len(communities)}")
print(f"  Temporal patterns: {len(temporal_patterns)}")


## Step 6: Analyze Market Patterns

Analyze real-time market patterns using temporal queries and pattern detection.


In [ ]:
# Temporal analysis
start_time = (datetime.now() - timedelta(hours=1)).isoformat()
end_time = datetime.now().isoformat()

temporal_results = temporal_query.query_time_range(
    graph=trading_kg,
    query="Find price movements in the last hour",
    start_time=start_time,
    end_time=end_time
)

# Inference engine for trading rules
inference_engine = InferenceEngine()
rule_manager = RuleManager()
explanation_generator = ExplanationGenerator()

# Trading analysis rules
inference_engine.add_rule("IF change_percent > 2 AND volume > 40000000 THEN strong_momentum")
inference_engine.add_rule("IF change_percent < -1 AND volume > 50000000 THEN selling_pressure")
inference_engine.add_rule("IF spread < 0.05 AND volume > 30000000 THEN high_liquidity")

# Add facts from real-time price data
for price_data in real_time_prices:
    if isinstance(price_data, dict):
        inference_engine.add_fact({
            "symbol": price_data.get("symbol", ""),
            "change_percent": price_data.get("change_percent", 0),
            "volume": price_data.get("volume", 0),
            "spread": price_data.get("spread", 0)
        })

# Generate trading insights
trading_insights = inference_engine.forward_chain()

print(f"✓ Temporal analysis completed")
print(f"  Temporal entities: {len(temporal_results.get('entities', []))}")
print(f"  Trading insights: {len(trading_insights)}")

# Display insights
for insight in trading_insights[:3]:
    print(f"  - {insight}")


## Step 7: Export and Visualize

Export the trading knowledge graph and generate visualizations.


In [ ]:
import tempfile
import os

temp_dir = tempfile.mkdtemp()

json_exporter = JSONExporter()
csv_exporter = CSVExporter()
rdf_exporter = RDFExporter()
report_generator = ReportGenerator()

# Export knowledge graph
json_exporter.export_knowledge_graph(trading_kg, os.path.join(temp_dir, "trading_kg.json"))
csv_exporter.export_entities(trading_entities, os.path.join(temp_dir, "trading_entities.csv"))
rdf_exporter.export_knowledge_graph(trading_kg, os.path.join(temp_dir, "trading_kg.rdf"))

# Generate report
report_data = {
    "summary": f"Real-time market data integration from MCP server identified {len(trading_insights)} insights",
    "price_points": len([e for e in trading_entities if e['type'] == 'PricePoint']),
    "stocks": len([e for e in trading_entities if e['type'] == 'Stock']),
    "market_events": len([e for e in trading_entities if e['type'] == 'MarketEvent']),
    "temporal_patterns": len(temporal_patterns),
    "insights": len(trading_insights)
}

report = report_generator.generate_report(report_data, format="markdown")

print("✓ Exported trading knowledge graph")
print(f"  JSON: {os.path.join(temp_dir, 'trading_kg.json')}")
print(f"  CSV: {os.path.join(temp_dir, 'trading_entities.csv')}")
print(f"  RDF: {os.path.join(temp_dir, 'trading_kg.rdf')}")
print(f"✓ Generated report ({len(report)} characters)")

# Visualize
kg_visualizer = KGVisualizer()
temporal_visualizer = TemporalVisualizer()
analytics_visualizer = AnalyticsVisualizer()

kg_viz = kg_visualizer.visualize_network(trading_kg, output="interactive")
temporal_viz = temporal_visualizer.visualize_timeline(trading_kg, output="interactive")
analytics_viz = analytics_visualizer.visualize_analytics(trading_kg, output="interactive")

print("✓ Generated visualizations for trading knowledge graph")

# Cleanup: Disconnect from MCP server
try:
    mcp_ingestor.disconnect("market_server")
    print("\n✓ Disconnected from MCP server")
except:
    pass

print(f"\n✅ Pipeline complete: MCP Server → Ingest Real-Time Data → Parse → Extract → Build Temporal KG → Analyze Patterns → Export → Visualize")
print(f"📊 Total modules used: 20+")
